# Summary

This notebook demonstrates an example to programmatically update an existing window recipe. This makes use of the DSS Python API `dataiku` (and is read inside of DSS). Read the [documentation](https://doc.dataiku.com/dss/latest/python-api/recipes.html) for more reference (particularly on recipe settings).

Firstly read in all the necessary packages - noting that visual recipes operate on json payloads (hence the `json` is required, to better deal with updating the metadata).

In [1]:
import dataiku
import pandas as pd
import json

## Setup

Firstly set up connections to the project, and the client.

In [2]:
client = dataiku.api_client()
project = client.get_project(dataiku.get_custom_variables()["projectKey"])

## Connect Recipe

In this example, change the text field in the `get_recipe` function to the window recipe on your Flow. There are programmatic ways to create recipes as well but will not be covered in this example.
Ensure you change the field below (`compute_txn_prep_windows`) to point to your recipe (case sensitive).

In [3]:
# change 'compute_txn_prep_windows' to point to your window recipe
windows_recipe = project.get_recipe("compute_txn_prep_windows")

The best way to observe what functions/classes/objects are available to the object is to hit 'tab' after typing out the object. In this example we obtain the recipe settings.

In [4]:
settings = windows_recipe.get_settings()

Finally, we inspect the JSON payload via the `get_payload` function from the `settings` object we have just created, then load it into the Python JSON object.

In [5]:
# load the settings payload into a json object, this will make it into a dictionary
payload_json = json.loads(settings.get_payload())

The above becomes a dictionary of objects; to identify what keys are available use: 

`payload_json.keys()`

In the example below we inspect the `windows` key for this recipe:

In [6]:
payload_json['windows']

[{'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  'precedingRows': 7,
  'prefix': 'L7',
  'windowLimitMode': 'ROWS',
  'windowLowerBound': 7,
  'windowUpperBound': -1}]

to which we can see there is only 1 window setting at the moment. We can programmatically update this in a repetitive manner.

# Updating JSON payload programmatically

The example below looks at iteratively simulating the creation of a L7, L14, ... window to apply to the respective aggregations.

Note that this can also be applied to a date (but in this example we chose not to, as there were PostgreSQL recipe engine issues).

In this example we look at expanding the windows in 4 iterations.

In [7]:
# specify interval and keys
intervals = range(3)

We now look for the keys we want to update iteratively.

In [8]:
# keys from the payload
# payload_json.keys()

keys = ['windowLowerBound', 'precedingRows', 'prefix']

Make a copy of the existing window so that we can iterate from this.

In [9]:
# as starting point, copy the dictionary from the first window
window_dict = payload_json['windows'][0]

Finally, run a loop to update the windows programmatically

In [10]:
# create a new dictionary and append it to the json payload
for n in intervals:
    # curr interval
    curr_int = (n + 2)* 7 # weekly
    # update values
    curr_values = (curr_int, curr_int, 'L' + str(curr_int))
    # take a copy of the dictionary
    temp_dict = window_dict.copy()

    # update the windows
    temp_dict.update(dict(zip(keys, curr_values)))
    
    # append it to the payload_json_windows
    payload_json['windows'].append(temp_dict)

inspect the payload to confirm

In [11]:
payload_json['windows']

[{'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  'precedingRows': 7,
  'prefix': 'L7',
  'windowLimitMode': 'ROWS',
  'windowLowerBound': 7,
  'windowUpperBound': -1},
 {'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  'precedingRows': 14,
  'prefix': 'L14',
  'windowLimitMode': 'ROWS',
  'windowLowerBound': 14,
  'windowUpperBound': -1},
 {'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  

## Updating the recipe settings

This final step loads in the JSON payload into the recipe. 

We firstly update the recipe settings.

In [14]:
settings.set_json_payload(payload_json)

Check that we have updated the `windows` payload:

In [15]:
settings.get_json_payload()['windows']

[{'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  'precedingRows': 7,
  'prefix': 'L7',
  'windowLimitMode': 'ROWS',
  'windowLowerBound': 7,
  'windowUpperBound': -1},
 {'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  'precedingRows': 14,
  'prefix': 'L14',
  'windowLimitMode': 'ROWS',
  'windowLowerBound': 14,
  'windowUpperBound': -1},
 {'enableLimits': True,
  'enableOrdering': True,
  'enablePartitioning': True,
  'followingRows': -1,
  'limitFollowing': True,
  'limitPreceding': True,
  'orders': [{'column': 'purchase_date', 'desc': False}],
  'partitioningColumns': ['card_id'],
  

and finally, we upload it back into the visual recipe (and check the visua

In [16]:
windows_recipe.set_definition_and_payload(settings)

/Users/mphang/Library/DataScienceStudio/kits/dataiku-dss-10.0.7-osx/python/dataikuapi/dss/recipe.py:172: DeprecationWarning: Recipe.set_definition_and_payload is deprecated, please use get_settings
  warnings.warn("Recipe.set_definition_and_payload is deprecated, please use get_settings", DeprecationWarning)


{'msg': 'Updated recipe DKU_TUT_MLOPS.compute_txn_prep_windows'}

That's it!